# Latar Belakang
Airbnb sedang melakukan pemeringkatan terhadap seluruh rumah yang terdaftar pada layanan mereka.  Mereka melakukannya demi bagusnya citra mereka bagi calon pelanggan jasa mereka.  Mereka ingin melakukan pemeringkatan dengan seberapa ramai pengunjung yang datang ke suatu rumah

## Pernyataan Masalah
airbnb ingin mengetahui **rumah mana yang ramai dikunjungi pengunjung**.  Kita dapat melakukannya dengan data jumlah review yang diterima oleh suatu rumah tersebut.  Kita sebagai data analyst berusaha menjawab pertanyaan **rumah mana yang ramai dikunjungi pengunjung** dengan menggunakan data yang tersedia.

# Data
Untuk menjawab pertanyaan di atas, kita akan menganalisa data peserta yang sudah dikumpulkan oleh perusahaan. Dataset dapat diakses [di sini](https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists?taskId=3015).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import normaltest, chi2_contingency, mannwhitneyu, ttest_ind

import warnings
warnings.filterwarnings("ignore")

Dataset ini berisi informasi terkait letak, kepemilikan, jumlah review, frekuensi review, kapan review yang terakhir kali dari seluruh rumah yang terdaftar pada layanan airbnb. Ada 16 kolom di dalam dataset airbnb_listings, yaitu:  

* id : ID unik airbnb untuk listing
* name: nama untuk listing
* host_id : ID unik airbnb untuk host rumah
* host_name: nama dari host rumah, biasanya hanya nama depan saja
* neighborhood: nama daerah
* latitude: garis lintang posisi rumah
* longitude: garis bujur posisi rumah
* room_type: jenis rumah
* price: harga rumah
* minimum_nights: lama minimum untuk tinggal di rumah tersebut
* number_of_reviews: jumlah review yang diberikan pengunjung ke rumah tersebut
* last_review: tanggal review terbaru
* reviews_per_month: frekuensi review suatu rumah per bulannya
* calculated_host_listings_count: rumah yang dimiliki oleh seorang host pada daerah yang sama
* availability_365: perkiraan berapa hari suatu rumah akan kosong
* number_of_reviews_ltm: jumlah review yang diberikan pengunjung ke rumah tersebut pada 12 bulan terakhir


Berikut 5 baris teratas dan terbawah dari dataset airbnb_listings.

In [ ]:
df=pd.read_csv('airbnb_listings.csv')
display(df.head(),df.tail())

FileNotFoundError: ignored

## Data Understanding and Cleaning
Sebelum masuk ke dalam analisis, kita perlu mengenal dataset kita lebih jauh dalam tahapan *data understanding*. Dari proses ini, kita akan tahu anomali-anomali apa saja yang terdapat di dalam dataset kita dan perlu ditangani dalam tahapan *data cleaning*. Setiap penangan anomali yang dilakukan, akan disertai dengan justifikasi langkah yang diambil, baik secara *domain knowledge* maupun secara statistik.

Pertama, mari kita lihat informasi umur dari dataset hr_analytics.

In [ ]:
print(f'Jumlah baris dan kolom di dataset df adalah {df.shape}')
df.info()

NameError: ignored

In [18]:
display(df.describe(), df.describe(include='object'))

NameError: ignored

In [19]:
pd.set_option('display.max_colwidth', -1)
# data uni di tiap kolom
listItem = []
for col in df.columns :
    listItem.append( [col, df[col].nunique(), df[col].unique()])

tabel1Desc = pd.DataFrame(columns=['Column Name', 'Number of Unique', 'Unique Sample'],
                     data=listItem)
tabel1Desc

NameError: ignored

Secara umum, kita bisa melihat bahwa:
* dataset airbnb_listings memiliki 16 kolom
* beberapa kolom memiliki data kosong. Data kosong pada kolom-kolom tersebut diwakili dengan data NaN.
* beberapa data kategorik ordinal, bisa dibuatkan kolom baru berdasarkan ranking kategorinya untuk memudahkan analisis (opsional)

### Missing Value

In [20]:
# persentase missing value di tiap kolom
df.isna().sum()/df.shape[0]*100

NameError: ignored

In [21]:
# Sebaran missing value di dalam data
plt.figure(figsize=(10,10))
sns.heatmap(df[['id', 'name', 'host_id', 'host_name', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'number_of_reviews_ltm']].isna())

NameError: ignored

<Figure size 1000x1000 with 0 Axes>

Secara garis besar:  
* *missing value* di beberapa kolom memiliki proporsi yang cukup tinggi.

Ada 2 cara untuk menangani *missing value*:
* pertama, menghapus baris/kolom yang berisi *missing value*. Cara ini tidak disarankan dalam kasus kita, karena jumlah missing value yang tergolong cukup tinggi (sampai 32%).
* kedua, mengisi data yang hilang. Cara ini lebih disarankan. Ada beberapa metode yang bisa digunakan untuk mengisi missing value, cara yang paling baik adalah dengan mengisi data yang hilang dengan nilai sebenarnya, atau sedekat mungkin dengan nilai asli. Dalam kasus ini, kita akan mencoba mengisi *missing value* berdasarkan kolom lain yang secara domain knowledge atau secara statistik berkaitan dengan kolom yang memiliki *missing value*. Jika masih ada kolom yang tidak bisa diisi, barulah kita mengisi dengan angka *mean, median* atau *modus*. Menghapus data akan menjadi opsi terakhir.

## Data yang sudah bersih
Kita sudah selesai membersihkan data kita, sekarang, mari kita lihat sekilas informasi dari data yang sudah kita bersihkan

In [22]:
listItem = []
for col in df.columns :
    listItem.append([col, df[col].dtype, df[col].isna().sum(),
                    df[col].nunique(), list(df[col].drop_duplicates().sample(2).values)]);

dfDesc = pd.DataFrame(columns=['dataFeatures', 'dataType', 'null', 'unique', 'uniqueSample'],
                     data=listItem)
print(df.shape)
dfDesc

NameError: ignored

Setelah dibersihkan beberapa data mengalami penghapusan dan sebagian yang lain mengalami perubahan

# Data Analysis
Kita sudah melakukan tahap _data cleaning_. Sekarang, kita bisa mulai melakukan analisis untuk mencari tahu **bagaimana cara menentukan rumah yang ramai**.  

# Kesimpulan dan Rekomendasi

Dari analisis yang telah dilakukan, kita bisa membuat kesilpulan berikut tentang rumah yang ramai :
* rumah yang ramai memiliki frekuensi review yang sering dengan tanggal review terbarunya yang tidak terlalu jauh

Dengan pemeringkatan ini, diharapkan analisis ini bisa membantu pengunjung rumah mana yang akan dipilih untuk mereka tinggali.